## Import

In [43]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import json
import matplotlib.pyplot as plt
from IPython.display import display
import os
import numpy as np
import sys 
sys.path.append('../../modules')
import data
import eval
from sklearn import preprocessing
from sklearn import pipeline
from sklearn import dummy
from sklearn import compose
from sklearn import multioutput
from sklearn import linear_model
from sklearn import ensemble
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import base

pd.set_option('display.min_rows', 20)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Data

In [3]:
values = data.load_migration_rates_from_csv()
trends = data.load_trends_from_csv()
countries = data.get_countries()
keyword_ids = trends.columns.levels[0].tolist()


# panel = pd.concat([values, trends], axis='columns')
panel = values.join(trends, how="outer")
#panel


## Models

### Naive: constant

In [4]:
t_min = "2012-01-01"
t_max = "2019-12-01"
t_split_lower = "2017-12-01"
t_split_upper = "2018-01-01"

panel_resampled = panel.resample("3M").mean()
panel_lags = data.create_lags(panel_resampled, lags=[1], columns=["value"]).fillna(0.0)

# define x, y; set minimum value
x = panel_lags.applymap(lambda x: max(x, 1.0))
y = panel_resampled["value"].applymap(lambda x: max(x, 1.0))

# transformation: logdiff
x_trans = np.log(x) - np.log(x.shift(4))
y_trans = np.log(y) - np.log(y.shift(4))

x_train = x_trans[t_min:t_split_lower]
y_train = y_trans[t_min:t_split_lower]
x_test = x_trans[t_split_upper:t_max]
y_test = y_trans[t_split_upper:t_max]

# stacking
x_train_stacked, y_train_stacked = data.stack(x_train, y_train, extra_column=False)
x_test_stacked, y_test_stacked = data.stack(x_test, y_test, extra_column=False)

cv = model_selection.KFold(n_splits=6, shuffle=False)

# ct = compose.make_column_transformer(
#     (preprocessing.OneHotEncoder(), ["country"]),
#     remainder="passthrough",
#     sparse_threshold=0,
# )

# reg = pipeline.make_pipeline(ct, linear_model.LinearRegression())
reg = dummy.DummyRegressor(strategy="constant", constant=0)

reg.fit(x_train_stacked, y_train_stacked)
# reg = hptuner.best_estimator_
# display(hptuner.best_params_)


cv_score = eval.score_cv(reg, x_train_stacked, y_train_stacked, cv=cv)
test_score = eval.score_test(reg, x_test_stacked, y_test_stacked)

print("Aggregate CV score:")
display(eval.agg_cv_scores(cv_score, use_quantiles=False))

print("Individual CV scores:")
display(cv_score)

print("Holdout score:")
display(test_score)


Aggregate CV score:


,mean,std,sem
fit_time,0.001,6.059e-04,2.474e-04
score_time,0.002,3.930e-04,1.604e-04
test_mae,-0.125,5.159e-02,2.106e-02
test_rmse,-0.178,7.521e-02,3.070e-02
test_explained_variance,0.000,0.000e+00,0.000e+00
test_r2_mod,0.000,0.000e+00,0.000e+00


Individual CV scores:


,fit_time,score_time,test_mae,test_rmse,test_explained_variance,test_r2_mod
0,1.336e-03,0.003,-0.221,-0.299,0.0,0.0
1,2.210e-03,0.002,-0.118,-0.189,0.0,0.0
2,6.995e-04,0.002,-0.136,-0.226,0.0,0.0
3,8.242e-04,0.002,-0.086,-0.114,0.0,0.0
4,7.854e-04,0.002,-0.106,-0.135,0.0,0.0
5,6.363e-04,0.003,-0.079,-0.105,0.0,0.0


Holdout score:


mae                  -0.121
rmse                 -0.301
explained_variance    0.000
r2_mod                0.000
dtype: float64

### Naive: previous

In [57]:
class LinearPredictionModel(linear_model.LinearRegression):
    """
    This model is for prediction only.  It has no fit method.
    You can initialize it with fixed values for coefficients 
    and intercepts.  

    Parameters
    ----------
    coef, intercept : arrays
        See attribute descriptions below.

    Attributes
    ----------
    coef_ : array of shape (n_features, ) or (n_targets, n_features)
        Coefficients of the linear model.  If there are multiple targets
        (y 2D), this is a 2D array of shape (n_targets, n_features), 
        whereas if there is only one target, this is a 1D array of 
        length n_features.
    intercept_ : float or array of shape of (n_targets,)
        Independent term in the linear model.
    """

    def __init__(self, coef=None, intercept=None):
        self.intercept = intercept
        self.coef = coef
        if coef is not None:
            coef = np.array(coef)
            if intercept is None:
                intercept = np.zeros(coef.shape[0])
            else:
                intercept = np.array(intercept)
            assert coef.shape[0] == intercept.shape[0]
        else:
            if intercept is not None:
                raise ValueError("Provide coef only or both coef and intercept")
        self.intercept_ = intercept
        self.coef_ = coef

    def fit(self, X, y):
        ...
        # """This model does not have a fit method."""
        # raise NotImplementedError("model is only for prediction")


t_min = "2012-01-01"
t_max = "2019-12-01"
t_split_lower = "2017-12-01"
t_split_upper = "2018-01-01"

panel_resampled = panel.resample("3M").mean()
panel_lags = data.create_lags(panel_resampled, lags=[1], columns=["value"]).fillna(0.0)

# define x, y; set minimum value
x = panel_lags.applymap(lambda x: max(x, 1.0))
y = panel_resampled["value"].applymap(lambda x: max(x, 1.0))

# transformation: logdiff
x_trans = np.log(x) - np.log(x.shift(4))
y_trans = np.log(y) - np.log(y.shift(4))

x_train = x_trans[t_min:t_split_lower]
y_train = y_trans[t_min:t_split_lower]
x_test = x_trans[t_split_upper:t_max]
y_test = y_trans[t_split_upper:t_max]

# stacking
x_train_stacked, y_train_stacked = data.stack(x_train, y_train, extra_column=False)
x_test_stacked, y_test_stacked = data.stack(x_test, y_test, extra_column=False)

cv = model_selection.KFold(n_splits=6, shuffle=False)

# reg = pipeline.make_pipeline(ct, linear_model.LinearRegression())
reg = LinearPredictionModel(np.array([1.0]), np.array([0.0]))

cv_score = eval.score_cv(reg, x_train_stacked, y_train_stacked, cv=cv)
test_score = eval.score_test(reg, x_test_stacked, y_test_stacked)

print("Aggregate CV score:")
display(eval.agg_cv_scores(cv_score, use_quantiles=False))

print("Individual CV scores:")
display(cv_score)

print("Holdout score:")
display(test_score)


Aggregate CV score:


/home/stei509/anaconda3/envs/eumf/lib/python3.9/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but LinearPredictionModel was fitted without feature names
  warnings.warn(
/home/stei509/anaconda3/envs/eumf/lib/python3.9/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but LinearPredictionModel was fitted without feature names
  warnings.warn(
/home/stei509/anaconda3/envs/eumf/lib/python3.9/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but LinearPredictionModel was fitted without feature names
  warnings.warn(
/home/stei509/anaconda3/envs/eumf/lib/python3.9/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but LinearPredictionModel was fitted without feature names
  warnings.warn(
/home/stei509/anaconda3/envs/eumf/lib/python3.9/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but LinearPredictionModel was fitted without feature names
  warnings.warn(
/home/stei509/anaconda3/envs/eumf/l

,mean,std,sem
fit_time,9.583e-04,4.393e-04,1.793e-04
score_time,3.818e-03,1.170e-03,4.777e-04
test_mae,-8.540e-02,1.341e-02,5.473e-03
test_rmse,-1.230e-01,2.110e-02,8.613e-03
test_explained_variance,2.967e-01,3.569e-01,1.457e-01
test_r2_mod,3.786e-01,3.543e-01,1.446e-01


Individual CV scores:


,fit_time,score_time,test_mae,test_rmse,test_explained_variance,test_r2_mod
0,1.736e-03,0.005,-0.100,-0.142,0.603,0.775
1,5.581e-04,0.003,-0.078,-0.129,0.431,0.532
2,1.187e-03,0.005,-0.085,-0.133,0.634,0.655
3,8.302e-04,0.003,-0.068,-0.088,0.409,0.413
4,6.263e-04,0.003,-0.103,-0.139,-0.112,-0.055
5,8.128e-04,0.003,-0.079,-0.108,-0.185,-0.048


Holdout score:


mae                  -0.132
rmse                 -0.360
explained_variance   -0.498
r2_mod               -0.431
dtype: float64